In [88]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [89]:
data = pd.read_csv('coins.csv')[['Name', 'Symbol', 'GitHub Link']]

In [90]:
github_links = data.iloc(1)[2]
sympols = data.iloc(1)[1]

In [91]:
driver = webdriver.Firefox()
driver.maximize_window()

In [92]:
coins_rank = list()
coins_contributor = list()
contributors_count = list()
for i in range(0, 5):
    contrbs = list()
    coins_rank.append(i) 
    if 'github.com' in github_links[i]:
        driver.get(github_links[i])
        try:
            time.sleep(2)
            driver.find_element(By.XPATH, '//*[@id="repository-container-header"]/div[1]/div[1]/div/span[1]/a').click()
        except:
            pass


        time.sleep(2)
        try:
            count_contrbs = int(driver.find_element(By.XPATH, '/html/body/div[1]/div[4]/main/div/header/div[2]/nav/div/ul/li[5]/a/span').text)
        except:
            count_contrbs = "No public countributors"
        contributors_count.append(count_contrbs)
        
        try:
            if "People" in driver.find_element(By.XPATH, '/html/body/div[1]/div[4]/main/div/header/div[2]/nav/div/ul/li[5]/a').text:
                driver.find_element(By.XPATH, "/html/body/div[1]/div[4]/main/div/header/div[2]/nav/div/ul/li[5]/a").click()
                time.sleep(2)
                contrb_elements = driver.find_elements(By.XPATH, '//*[@class="f4 d-block"]')
                for j in contrb_elements:
                    contrbs.append(j.get_attribute('href'))
            else:
                time.sleep(2)
                driver.find_element(By.XPATH, "/html/body/div[1]/div[4]/main/div/header/div[2]/nav/div/ul/li[6]/a").click() # li[5] changed to li[6]
                time.sleep(2)
                contrb_elements = driver.find_elements(By.XPATH, '//*[@class="f4 d-block"]')
                for j in contrb_elements:
                    contrbs.append(j.get_attribute('href'))
        except:
            pass
        
    else:
        contrbs = ['No GitHub Link']
        
    coins_contributor.append(contrbs)



In [48]:
coins_rank = [i + 1 for i in coins_rank]

In [78]:
coins_repos = list()
coins_lang = list()

for i in range(0, 5):
    repos = list()
    langs = list()
    if 'github' in github_links[i]:        
        driver.get(github_links[i])
        time.sleep(2)
        try:
            driver.find_element(By.XPATH, '//*[@id="repository-container-header"]/div[1]/div[1]/div/span[1]/a').click()
        except:
            pass

        time.sleep(3)

        top_languages = driver.find_elements(By.XPATH, '//span[@itemprop="programmingLanguage"]')
        for language in top_languages:
            langs.append(language.text)


        coins_lang.append(langs)

        driver.find_element(By.XPATH, '//*[@id="sort-options"]').click()
        driver.find_element(By.XPATH, '//*[@id="sort-options"]/details-menu/div/div/label[3]').click()
        driver.find_element(By.XPATH, '//*[@id="type-options"]/summary').click()
        driver.find_element(By.XPATH, '//*[@id="type-options"]/details-menu/div/div/label[3]').click()
        time.sleep(2)
        

        time.sleep(2)
        repo_rows = driver.find_elements(By.XPATH, "//*[@class='Box-row']")
        # the following scripts loop through every repository row 
        for element in repo_rows:

            repo_name = element.find_element(By.XPATH, ".//*[@class='Link f4 d-inline-block text-bold mr-1']").text 
            repo_url = element.find_element(By.XPATH, ".//*[@class='Link f4 d-inline-block text-bold mr-1']").get_attribute('href')
            try:
                repo_detail = element.find_element(By.XPATH, ".//*[@class='color-fg-muted mb-3 wb-break-word']").text
            except:
                repo_detail = 'Repository has no detail'

            # the followin scripst use unique icones to reach the elements text
            try:
                licence_icon = element.find_element(By.XPATH, ".//*[@class='octicon octicon-law mr-1']")
                licence = licence_icon.find_element(By.XPATH, '..').text
            except:
                licence = 'No Licence'

            stars_icon = element.find_element(By.XPATH, ".//*[@class='octicon octicon-star mr-1']")
            stars = stars_icon.find_element(By.XPATH, '..').text

            forks_icon = element.find_element(By.XPATH, ".//*[@class='octicon octicon-repo-forked mr-1']")
            forks = forks_icon.find_element(By.XPATH, '..').text


            repository = [  sympols[i],
                            repo_name,
                            repo_url,
                            repo_detail,
                            licence,
                            stars,
                            forks]
                
            repos.append(repository)
    else:
        coins_repos.append([[sympols[i], 'No GitHub Link']])
        coins_lang.append('No GitHub Link')

    coins_repos.append(repos)


In [ ]:
column_names = [
    "Rank",
    "Counterbutors Count",
    "Counterbutors Url"
]
coins_countributors_df = pd.DataFrame(list(zip(coins_rank, contributors_count, coins_contributor)), columns=column_names)
coins_countributors_df


In [ ]:
coins_countributors_df.to_csv('../datasets/coins_countributors.csv' , index = False , header = True)

In [ ]:
column_names = [
    "Rank",
    "Languages"
]
coins_lang_df = pd.DataFrame(list(zip(coins_rank, coins_lang)), columns=column_names)
coins_lang_df

In [ ]:
coins_lang_df.to_csv('../datasets/coins_language.csv', index = False , header = True)

In [ ]:
flat_data = [repo for sublist in coins_repos for repo in sublist]
column_names = [
    'Symbol',
    'Name',
    'Url',
    'Detail',
    'Licence',
    'Stars',
    'Forks'
]
coins_repositories_df = pd.DataFrame(flat_data, columns=column_names)
coins_repositories_df

In [ ]:
coins_lang_df.to_csv('../datasets/coins_repositories.csv', index = False , header = True)